In [1]:
import sys, os
software = "/data/shared/Software/"
if(not software in sys.path):
    sys.path.append(software)
p = "/home/dweitekamp/mpi_learn/"
if(not p in sys.path):
    sys.path.append(p)
from CMS_SURF_2016.utils.preprocessing import *
from CMS_SURF_2016.utils.callbacks import OverfitStopping, SmartCheckpoint
from CMS_SURF_2016.utils.batch import batchAssertArchived, batchExecuteAndTestTrials
from CMS_SURF_2016.utils.archiving import *
from CMS_SURF_2016.utils.analysistools import findsubsets
from CMS_SURF_2016.layers.lorentz import Lorentz, _lorentz
from CMS_SURF_2016.layers.slice import Slice


Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
# from CMS_SURF_2016.utils.archiving import KerasTrial, DataProcedure
from CMS_SURF_2016.utils.batch import batchAssertArchived
#import mpi_learn

import sys,os
import numpy as np
# import argparse
import json
import shlex
import subprocess
from mpi4py import MPI
from time import time,sleep
import select
import importlib
#import psutil


In [3]:
import mpi_learn

In [4]:
sort_on = "PT_ET"
observ_types = ['E/c', 'Px', 'Py', 'Pz', 'PT_ET','Eta', 'Phi', 'Charge', 'X', 'Y', 'Z',\
                     'Dxy', 'Ehad', 'Eem', 'MuIso', 'EleIso', 'ChHadIso','NeuHadIso','GammaIso', 'ObjType']
vecsize = len(observ_types)
epochs = 100
batch_size = 100
label_dir_pairs = \
            [   ("ttbar", "/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_h5/"),
                ("wjet", "/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_h5/"),
                ("qcd", "/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_h5/")
            ]

object_profiles = [ObjectProfile("Electron",-1, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":1}),
                            ObjectProfile("MuonTight", -1, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":2}),
                            #ObjectProfile("Photon", -1, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":3}),
                            ObjectProfile("MissingET", 1, addColumns={"ObjType":4}),
                            ObjectProfile("EFlowPhoton",100, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":5}), 
                            ObjectProfile("EFlowNeutralHadron",100, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":6}), 
                            ObjectProfile("EFlowTrack",100, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":7})]  

resolveProfileMaxes(object_profiles, label_dir_pairs)

def genModel(name,object_profiles,out_dim, depth, width, dense_activation="relu", dropout = 0.0,sphereCoords=True,weight_output=False):
    inputs = []
    mergelist = []
    for i, profile in enumerate(object_profiles):
        # print(o)
        inp = a = Input(shape=(profile.max_size, vecsize), name="input_"+str(i))
        inputs.append(inp)

        if(name == 'lorentz'):
            b1 = Lorentz(sphereCoords=sphereCoords, weight_output=weight_output ,name="lorentz_"+str(i))(a)
            b1 = Flatten(name="flatten1_"+str(i))(b1)
        elif(name == 'control_dense'):
            b1 = Slice('[:,0:4]',name='slice_1_'+str(i))(a)
            b1 = Flatten(name="4_flatten_"+str(i))(b1)
            b1 = Dense(4 * profile.max_size, activation='linear', name='4_dense_'+str(i))(b1)
        else:
            b1 = Slice('[:,0:4]',name='slice_1_'+str(i))(a)
            b1 = Flatten(name="flatten1_"+str(i))(b1)

        
        b2 = Slice('[:,4:]',name='slice_2_'+str(i))(a)
        b2 = Flatten(name="flatten_2_"+str(i))(b2)
        # b2 = Dense(10, activation='relu')(b2)
        mergelist.append(b1)
        mergelist.append(b2)
    # print(mergelist)
    a = merge(mergelist,mode='concat', name="merge")
    # a = Flatten()(a)
    for i in range(depth):
        a =  Dense(width, activation=dense_activation, name="dense_"+str(i))(a)
        if(dropout > 0.0):
            a =  Dropout(dropout, name="dropout_"+str(i))(a)
    dense_out = Dense(out_dim, activation='sigmoid', name='main_output')(a)
    model = Model(input=inputs, output=dense_out, name=name)
    return model

In [15]:
#%matplotlib inline
import sys, os
if __package__ is None:
	import sys, os
	sys.path.append(os.path.realpath("/data/shared/Software/"))
	sys.path.append(os.path.realpath("../.."))

from CMS_SURF_2016.utils.archiving import *
from CMS_SURF_2016.utils.MPIArchiving import *

from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Reshape, Activation, Dropout, Convolution2D, Merge, Input
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical

from keras.layers import Dense, Flatten, Reshape, Activation, Dropout, Convolution2D, merge, Input, Flatten, Lambda, LSTM, Masking
from keras.engine.topology import Layer
import numpy as np


archive_dir = '/data/shared/Delphes/keras_archive/'

#Define callback
earlystopping = EarlyStopping(patience=10, verbose=1)

#Make two input branches
#left_branch = Sequential()
#left_branch.add(Dense(32, input_dim=784))#, name=namer.get('dense')))
#right_branch = Sequential()
#right_branch.add(Dense(32, input_dim=784))#, name=namer.get('dense')))
#merged = Merge([left_branch, right_branch], mode='concat')#, name=namer.get('merge'))

#Make two layer dense model
#model = Sequential()
#model.add(merged)
#model.add(Dense(10, activation='softmax'))#, name=namer.get('dense')))
#model.add(Dense(10, activation='softmax'))#, name=namer.get('dense')))
#for prof in object_profiles:
#    print(prof.__dict__)
#print(object_profiles)

some_trial = KerasTrial.find_by_hashcode(archive_dir,"381915c32564add6ccfdebf7d3ca6a4d6d8efae6")
model = genModel("lorentz",object_profiles, 3, 1,10)
#model = some_trial.compile()
train_dps = DataProcedure.from_json(archive_dir,some_trial.train_procedure[0]).args[0]
val_dps = DataProcedure.from_json(archive_dir,some_trial.val_procedure[0]).args[0]
print(train_dps)
print(val_dps)
print(train_dps[0].getData())
raise ValueError()
#print(some_trial.train_procedure)
print()
#train_dps = some_trial.train_procedure[0].args[0]
#val_dps = some_trial.val_procedure[0].args[0]
#

#trial = MPI_KerasTrial(archive_dir,workers=2, name="MyKerasTrial", model=model, custom_objects={"Lorentz": Lorentz, "Slice" : Slice})
trial = KerasTrial(archive_dir, name="MyKerasTrial", model=model)
#Set the training data
#trial.setTrain(train_procedure=train_dps)
#trial.setValidation(val_dps)
trial.setTrain(train_procedure=some_trial.train_procedure)
trial.setValidation(val_procedure=some_trial.val_procedure,nb_val_samples=some_trial.nb_val_samples)
#Set the compilation paramters
trial.setCompilation(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )
#Set the fit parameters
trial.setFit(callbacks = [earlystopping], nb_epoch=18)

print(model.summary())
#print(trial.model.summary())
print(trial.hash())
#If we like we can store information about the trial so we can keep track of what we've done
trial.to_record({"stuff1":100, "data_stuff": "Regular"})

print(trial.hash())
trial.to_hashable()
# raise ValueError()


trial.execute(custom_objects={"Lorentz": Lorentz, "Slice" : Slice})
#trial.remove_from_archive()
print("OK IT FINISHED!")


print(trial.hash())


[<CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c65d4d90>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c65d4ed0>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c6578a90>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c6578490>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c6578c90>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c6578e90>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c659d0d0>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c659d2d0>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c659d4d0>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c659d6d0>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c659d8d0>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c659dad0>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c659dcd0>, <CMS_SURF_2016.utils.archiving.DataProcedure object at 0x7f38c6

ValueError: 

In [ ]:
Algo("adadelta")

# 